# Part 1

Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:

```
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
```

...then you would be able to recreate the structure of the towers that looks like this:

```
                gyxo
              /     
         ugml - ebii
       /      \     
      |         jptl
      |        
      |         pbga
     /        /
tknk --- padx - havc
     \        \
      |         goyq
      |             
      |         ktlj
       \      /     
         fwft - cntj
              \     
                xhth
```

In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

In [1]:
import weakref

In [2]:
class ProgramNode:
    _nodes = {}
    
    def __init__(self, name, weight=None, parent=None, children=None):
        self.name = name
        self.weight = weight
        self.parent = parent
        self.children = weakref.WeakSet(children) or weakref.WeakSet()
    
    @classmethod
    def from_line(cls, line: str):
        if '->' in line:
            name, weight, _, children = line.split(maxsplit=3)
            children = children.strip().split(', ')
        else:
            name, weight = line.split()
            children = []
        
        weight = int(weight[1:-1])
        child_nodes = []
        for child in children:
            if child in cls._nodes:
                child_node = cls._nodes[child]
            else:
                child_node = cls(child)
                cls._nodes[child] = child_node
                
            child_nodes.append(child_node)
        
        if name in cls._nodes:
            node = cls._nodes[name]
            node.weight = weight
            node.children.update(child_nodes)
        else:
            node = cls(name, weight, children=child_nodes)
            cls._nodes[name] = node
        
        for n in child_nodes:
            n.parent = node
        
        return node
    
    def __hash__(self):
        return hash(self.name)
    
    def __repr__(self):
        s = self
        parent_name = s.parent.name if s.parent else None
        child_names = [n.name for n in self.children]
        return f'ProgramNode(name={s.name}, weight={s.weight}, parent={parent_name}, children={child_names})'
    
    def subtree_weight(self):
        return self.weight + sum(n.subtree_weight() for n in self.children)

In [3]:
def pzl_input(input_file='./inputs/day07/input.txt'):
    ProgramNode._nodes.clear()
    with open(input_file) as f:
        return [ProgramNode.from_line(line) for line in f]

In [4]:
def find_bottom(nodes: list):
    for n in nodes:
        if not n.parent:
            return n

In [5]:
result = find_bottom(pzl_input('./inputs/day07/test_input.txt'))
assert result.name == 'tknk', result.name

In [6]:
find_bottom(pzl_input())

ProgramNode(name=svugo, weight=32, parent=None, children=['gjxqx', 'xolvnpy', 'gtzxxav', 'njorjq', 'qpiklvf'])

# Part 2

The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.

In the example above, this means that for ugml's disc to be balanced, gyxo, ebii, and jptl must all have the same weight, and they do: 61.

However, for tknk to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same:

```
ugml + (gyxo + ebii + jptl) = 68 + (61 + 61 + 61) = 251
padx + (pbga + havc + qoyq) = 45 + (66 + 66 + 66) = 243
fwft + (ktlj + cntj + xhth) = 72 + (57 + 57 + 57) = 243
```

As you can see, tknk's disc is unbalanced: ugml's stack is heavier than the other two. Even though the nodes above ugml are balanced, ugml itself is too heavy: it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced. If this change were made, its weight would be 60.

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

In [7]:
from collections import Counter

In [8]:
def outlier_node(node):
    counts = Counter(n.subtree_weight() for n in node.children)
    if len(counts) <= 1:
        return node
    else:
        _, (outlier_weight, _) = counts.most_common()
        return outlier_node({n.subtree_weight(): n for n in node.children}[outlier_weight])

In [9]:
outlier_node(find_bottom(pzl_input('./inputs/day07/test_input.txt')))

ProgramNode(name=ugml, weight=68, parent=tknk, children=['ebii', 'gyxo', 'jptl'])

In [10]:
root_node = find_bottom(pzl_input())
for n in root_node.children:
    print(n.name, n.subtree_weight())

gjxqx 64661
xolvnpy 64652
gtzxxav 64652
njorjq 64652
qpiklvf 64652


In [11]:
outlier_node(find_bottom(pzl_input()))

ProgramNode(name=sphbbz, weight=1161, parent=yruivis, children=['oprfjt', 'vyufo', 'ivpbt', 'scetry', 'gioryg', 'valbm', 'sfjspc'])

In [12]:
1161 - 9

1152